In [ ]:
import os
import numpy as np
from temporalcontext import settings
from temporalcontext.functions import read_selmap, prepare_cnn_inputs

## Prepare input data for training CNN models by extracting audio segments corresponding to annotations and saving as spectrograms.

In [ ]:
# The root directory under which audio & corresponding annotation files are
# available
audio_root = os.path.join(settings.raw_data_root, settings.raw_audio_dir)
seltab_root = os.path.join(settings.raw_data_root, settings.raw_annot_dir)

selmap = read_selmap(os.path.join(settings.raw_data_root, 'selmap.csv'))

# Loop on unique segment_advance from experiments
for seg_adv in list(set([ex['segment_advance'] for ex in settings.lstm_experiments])):
    
    print('Segment advance: {:.2f}:'.format(seg_adv))
    
    output_root = os.path.join(settings.project_root,
                               settings.segments_dir,
                               'seg_adv_{:.2f}'.format(seg_adv))
    
    total_pos_count = 0
    total_neg_count = 0

    # Process each file
    for au_file, pos_annot_file, neg_annot_file in selmap:
        
        pos_count, neg_count = prepare_cnn_inputs(
            os.path.join(audio_root, au_file),
            os.path.join(seltab_root, pos_annot_file),
            os.path.join(seltab_root, neg_annot_file),
            settings.segment_length, seg_adv,
            os.path.join(output_root, settings.class_dirs[0], au_file + '.npz'),
            os.path.join(output_root, settings.class_dirs[1], au_file + '.npz'),
            settings.specgram_params, settings.bandwidth_extents
        )
        
        total_pos_count += pos_count
        total_neg_count += neg_count
        
    print('  Positive segments: {:5d}'.format(total_pos_count))
    print('  Negative segments: {:5d}'.format(total_neg_count))
